link for the material: https://nlp.stanford.edu/IR-book/pdf/05comp.pdf

In [ ]:

import numpy as np
import heapq
import struct

In [ ]:

def gamma_encode(num):
    if num == 0:
        return '0'
    binary = bin(num)[2:]
    unary = '1' * (len(binary) - 1)
    return unary + binary

def gamma_decode(bitstring):
    unary_len = bitstring.index('0') + 1
    binary = '1' + bitstring[unary_len:]
    return int(binary, 2)

def delta_encode(num):
    if num == 0:
        return '0'
    binary = bin(num)[2:]
    gamma = gamma_encode(len(binary))
    return gamma + binary[1:]

def delta_decode(bitstring):
    unary_len = bitstring.index('0') + 1
    gamma_len = unary_len + unary_len - 1
    binary = '1' + bitstring[unary_len:gamma_len]
    return int(binary, 2) << (len(bitstring) - gamma_len) | int(bitstring[gamma_len:], 2)

def vb_encode(num):
    if num == 0:
        return b'\x00'
    bytes_ = []
    while num > 0:
        bytes_.append(num % 128)
        num //= 128
    bytes_[-1] += 128
    return bytes(bytes_[::-1])

def vb_decode(bytestring):
    num = 0
    for byte in bytestring:
        num = num * 128 + (byte & 127)
        if byte & 128 == 0:
            break
    return num


In [ ]:

def compress_postings_list(postings_list, encoding='gamma'):
    if encoding == 'gamma':
        return ''.join(gamma_encode(postings_list[i] - postings_list[i - 1]) for i in range(1, len(postings_list)))
    elif encoding == 'delta':
        return ''.join(delta_encode(postings_list[i] - postings_list[i - 1]) for i in range(1, len(postings_list)))
    elif encoding == 'vb':
        return b''.join(vb_encode(postings_list[i] - postings_list[i - 1]) for i in range(1, len(postings_list)))

def decompress_postings_list(encoded_postings_list, encoding='gamma'):
    if encoding == 'gamma':
        return [0] + [heapq.nlargest(i, [gamma_decode(encoded_postings_list[j:j + i]) for j in range(0, len(encoded_postings_list), i)])[-1] for i in range(1, len(encoded_postings_list) + 1, 2)]
    elif encoding == 'delta':
        return [0] + [heapq.nlargest(i, [delta_decode(encoded_postings_list[j:j + i]) for j in range(0, len(encoded_postings_list), i)])[-1] for i in range(1, len(encoded_postings_list) + 1, 2)]
    elif encoding == 'vb':
        return [0] + [heapq.nlargest(i, [vb_decode(encoded_postings_list[j:j + i]) for j in range(0, len(encoded_postings_list), i)])[-1] for i in range(1, len(encoded_postings_list) + 1)]
